In [1]:
!pip install mtcnn
!pip install kaggle

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elin75/localized-audio-visual-deepfake-dataset-lav-df")

print("Path to dataset files:", path)

Resuming download from 1602224128 bytes (23240237406 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/elin75/localized-audio-visual-deepfake-dataset-lav-df?dataset_version_number=1 (1602224128/24842461534) bytes left.


100%|██████████| 23.1G/23.1G [02:32<00:00, 153MB/s] 

Extracting files...


Path to dataset files: /teamspace/studios/this_studio/.cache/kagglehub/datasets/elin75/localized-audio-visual-deepfake-dataset-lav-df/versions/1


In [5]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [imblearn]1/3 [imbalanced-learn]


In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import add, BatchNormalization, GlobalAveragePooling2D, LSTM, Dense, Dropout, Input, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input  # Or any other preprocessing based on your model
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

In [7]:
Datagen=ImageDataGenerator(rescale=1./255)

In [12]:
import os
import cv2
import json
from tqdm import tqdm

# Constants
VIDEO_ROOT = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/elin75/localized-audio-visual-deepfake-dataset-lav-df/versions/1/LAV-DF"
FRAME_OUTPUT_DIR = "LAV-DF-Frames"
METADATA_PATH = os.path.join(VIDEO_ROOT, "metadata.min.json")
FRAME_SIZE = (256, 256)
MAX_FRAMES = 30

# Load metadata
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# Select 50 test, 50 train, 50 dev
test_entries, train_entries, dev_entries = [], [], []

for entry in metadata:
    if entry["split"] == "test" and len(test_entries) < 800:
        test_entries.append(entry)
    elif entry["split"] == "train" and len(train_entries) < 2000:
        train_entries.append(entry)
    elif entry["split"] == "dev" and len(dev_entries) < 1000:
        dev_entries.append(entry)

    if len(test_entries) == 800 and len(train_entries) == 2000 and len(dev_entries) == 1000:
        break

selected_entries = test_entries + train_entries + dev_entries


# Frame extractor
def extract_and_save_frames(video_path, label, subset, video_id):
    cap = cv2.VideoCapture(video_path)
    count = 0
    output_dir = os.path.join(FRAME_OUTPUT_DIR, subset, label, video_id)
    os.makedirs(output_dir, exist_ok=True)

    while count < MAX_FRAMES:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, FRAME_SIZE)
        cv2.imwrite(os.path.join(output_dir, f"{count}.jpg"), frame)
        count += 1
    cap.release()

# Process selected entries
for entry in tqdm(selected_entries, desc="Extracting frames"):
    file_path = entry["file"]  # e.g., 'train/000001.mp4'
    subset = entry["split"]
    video_id = os.path.splitext(os.path.basename(file_path))[0]
    label = "fake" if entry["modify_video"] else "real"
    video_path = os.path.join(VIDEO_ROOT, file_path)

    extract_and_save_frames(video_path, label, subset, video_id)


Extracting frames: 100%|██████████| 3800/3800 [02:43<00:00, 23.28it/s]


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    "LAV-DF-Frames/train",
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

valid_gen = datagen.flow_from_directory(
    "LAV-DF-Frames/dev",
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

Found 60000 images belonging to 2 classes.
Found 30000 images belonging to 2 classes.


In [14]:
model = InceptionV3(
    include_top = False,
    weights = "imagenet",
    input_shape = (256,256,3)
)

for layer in model.layers:
  layer.trainable = False

# Get the output of the InceptionV3 model
x = model.output

# Add new layers on top of the InceptionV3 output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(128, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 128 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
x = Dense(64, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 64 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.2)(x)  # Dropout layer with 50% dropout rate
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation

# Create a new model with the specified inputs and outputs
model1 = Model(inputs=model.input, outputs=x)

model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 127, 127,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 127, 127,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 127, 127,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 125, 125,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 125, 125,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 62, 62,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 62, 62,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 62, 62,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 60, 60,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 60, 60,    │          0 │ batch_normalizat

 Total params: 22,074,145 (84.21 MB)

 Trainable params: 270,977 (1.03 MB)

 Non-trainable params: 21,803,168 (83.17 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# import precision, Recall
from tensorflow.keras.metrics import Precision, Recall
# from tensorflow_addons.metrics import F1Score  # If using TensorFlow Addons

# Compile the model
model1.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="binary_crossentropy",
    metrics=["accuracy", Precision(), Recall()]
)
# Callbacks
earlystopping = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience = 200,
    restore_best_weights=True
)

modelcheckpoint = ModelCheckpoint(
    "InceptionV3ModelUpdated.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    verbose=1,
    min_lr=1e-6,
    patience = 1
)

# Training the model
history = model1.fit(
    train_gen,
    epochs=1000,
    validation_data=valid_gen,
    verbose=1,
    callbacks=[earlystopping, modelcheckpoint, reduce_lr]
)

Epoch 1/1000


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5310 - loss: 4.3079 - precision_1: 0.5339 - recall_1: 0.5338
Epoch 1: val_loss improved from inf to 0.76613, saving model to InceptionV3ModelUpdated.h5


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 268s 134ms/step - accuracy: 0.5310 - loss: 4.3066 - precision_1: 0.5339 - recall_1: 0.5339 - val_accuracy: 0.5010 - val_loss: 0.7661 - val_precision_1: 0.5021 - val_recall_1: 0.9688 - learning_rate: 5.0000e-04
Epoch 2/1000
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5330 - loss: 0.7399 - precision_1: 0.5407 - recall_1: 0.5621
Epoch 2: val_loss improved from 0.76613 to 0.73894, saving model to InceptionV3ModelUpdated.h5


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 239s 127ms/step - accuracy: 0.5330 - loss: 0.7399 - precision_1: 0.5407 - recall_1: 0.5621 - val_accuracy: 0.4975 - val_loss: 0.7389 - val_precision_1: 0.5095 - val_recall_1: 0.0250 - learning_rate: 5.0000e-04
Epoch 3/1000
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5513 - loss: 0.7064 - precision_1: 0.5657 - recall_1: 0.5031

In [ ]:
test_gen=Datagen.flow_from_directory(
    'LAV-DF-Frames/test',
    target_size=(256,256),
    batch_size=64,
    class_mode='binary',
    shuffle=True
)

In [ ]:
loss, accuracy, precision, recall = model.evaluate(testgen, verbose=1)